<a href="https://colab.research.google.com/github/StevenVuong/compvis_playground/blob/master/jupyter_notebooks/Image_Super_Resolution_Prediction_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Prediction_Tutorial.ipynb)

Credits to Idealo. References & Further Documentation:
-  https://github.com/idealo/image-super-resolution 
-  https://idealo.github.io/image-super-resolution/tutorials/docker/ 

### Fuse To GCS

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [0]:
!mkdir compvis_playground
!gcsfuse --implicit-dirs compvis_playground /content/compvis_playground

In [0]:
!ls compvis_playground/facenet

# Install ISR
(skip if using local repo)

In [0]:
!git clone https://github.com/idealo/image-super-resolution
%cd image-super-resolution
!python3 ./setup.py install

In [0]:
import os 
import numpy as np
from PIL import Image

from progressbar import ProgressBar

# Predict

## Get predictions

### Create the model and run prediction
Create a RRDN or RDN model with the provided pre-trained weights, choose one

In [0]:
# Uncomment if using local repo
# import sys
# sys.path.append('..')
from ISR.models import RDN, RRDN

weights_list = ["noise-cancel", "gans", "psnr-small", "psnr-large"]

# found to be best one for increasing face res / cancel noise
model = RDN(weights='noise-cancel') 

#### Alternatively, load your own weights
To load your own weights first build the model specifying the architecture parameters and then load the weights like with any Keras model.

In [0]:
# rdn = RDN(arch_params={'C': 5, 'D':16, 'G':48, 'G0':52, 'x':3})
# rdn.model.load_weights('PATH/TO/WEIGHTS')

#### Loop through Images; Predict and Save

In [0]:
# Also need to do test_aligned images
im_basepath = "/content/compvis_playground/facenet/images/test_aligned"
im_savepath = "/content/compvis_playground/facenet/images/test_aligned_isr"

pbar = ProgressBar()

for name_dir in pbar(os.listdir(im_basepath)):

  namedir_path = os.path.join(im_basepath, name_dir)

  # If not subdirectory, skip.
  if not os.path.isdir(namedir_path):
    continue

  for filename in os.listdir(namedir_path):

    # If not image file, skip.
    if not filename.endswith((".png", ".jpeg", ".jpg")):
      continue

    filepath = os.path.join(namedir_path, filename)

    print(f"Loading Image: {filepath}")
    img = Image.open(filepath)

    print(f"Predicting Super Res Image for {filename}")
    sr_img = model.predict(np.array(img))
    pred_img = Image.fromarray(sr_img)

    save_dir = os.path.join(im_savepath, name_dir)
    print(f"Saving Image to {save_dir}/")

    if not os.path.exists(save_dir):
      os.makedirs(save_dir)

    file_savepath = os.path.join(save_dir, filename)
    pred_img.save(file_savepath)

  #   break
  # break

#### Baseline

In [0]:
# img.resize(size=(img.size[0]*4, img.size[1]*4), resample=Image.BICUBIC)

#### Prediction

In [0]:
sr_img = model.predict(np.array(img))
pred_img = Image.fromarray(sr_img)

In [0]:
type(pred_img)